### Equação de Burgers'


$\dfrac{\partial u}{\partial t}+u\dfrac{\partial u}{\partial x}=\nu\dfrac{\partial^2 u}{\partial x^2}, \,\,\,\, x\in [-1,1], \,\,\,\, t\in [0,1]$

com as seguintes condições iniciais:

$u(-1,t) = u(1,t)=0,\,\,\,\,\, u(x,0)=-\sin(\pi x).$

In [ ]:
import deepxde as dde
from deepxde.backend import tf
import numpy as np

geom = dde.geometry.Interval(-1, 1)
timedomain = dde.geometry.TimeDomain(0, 0.99)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

def pde(x, y):
    dy_x = dde.grad.jacobian(y, x, i=0, j=0)
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    return dy_t + y * dy_x - 0.01 / np.pi * dy_xx

bc = dde.icbc.DirichletBC(geomtime, lambda x: 0, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(geomtime, lambda x: -np.sin(np.pi * x[:, 0:1]), lambda _, on_initial: on_initial)

data = dde.data.TimePDE(
    geomtime, pde, [bc, ic],
    num_domain=2540, num_boundary=80, num_initial=160, num_test=1500  # Ajuste para separar dados de teste
)
net = dde.nn.FNN([2] + [20] * 3 + [1], "tanh", "Glorot normal")
model = dde.Model(data, net)
model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(iterations=15000)
dde.saveplot(losshistory, train_state, issave=True, isplot=True) 
